## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
# Calculate the total count
# Calculate value counts for APPLICATION_TYPE
counts = application_df['APPLICATION_TYPE'].value_counts()

# Calculate the total count
total_count = counts.sum()

# Calculate the 95% threshold
threshold_95 = 0.95 * total_count

# Sort the value counts in descending order
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

# Calculate cumulative counts and find the threshold point
cumulative_count = 0
categories_to_keep = []
for category, count in counts.items():
    cumulative_count += count
    categories_to_keep.append(category)
    if cumulative_count >= threshold_95:
        break

# Print the result
print("Categories to Keep:")
for category in categories_to_keep:
    print(category)

Categories to Keep:
T3
T4
T6
T5
T19
T8


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Create a list of application types to be replaced
application_types_to_replace = list(set(counts.index) - set(categories_to_keep))

# Replace in the dataframe
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(lambda x: x if x in categories_to_keep else "Other")

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

T3       27037
T4        1542
Other     1529
T6        1216
T5        1173
T19       1065
T8         737
Name: APPLICATION_TYPE, dtype: int64


In [6]:
# Look at CLASSIFICATION value counts for binning
# Calculate the total count
# Calculate value counts for CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Calculate the total count
total_count = classification_counts.sum()

# Calculate the 95% threshold
threshold_95 = 0.95 * total_count

# Sort the value counts in descending order
sorted_counts = sorted(classification_counts.items(), key=lambda x: x[1], reverse=True)

# Calculate cumulative counts and find the threshold point
cumulative_count = 0
categories_to_keep = []
for category, count in classification_counts.items():
    cumulative_count += count
    categories_to_keep.append(category)
    if cumulative_count >= threshold_95:
        break

# Print the result
print("Categories to Keep:")
for category in categories_to_keep:
    print(category)

Categories to Keep:
C1000
C2000
C1200
C3000
C2100
C7000


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(set(counts.index) - set(categories_to_keep))

# Replace in the dataframe
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].apply(lambda x: x if x in categories_to_keep else "Other")

# Check to make sure binning was successful
print(application_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64


In [9]:
#scratch work
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [10]:
# Convert categorical data to numeric with pd.get_dummies
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
dummies = pd.get_dummies(application_df[categorical_columns])

# Drop the original categorical columns
application_df = application_df.drop(columns=categorical_columns)

# Concatenate dummy variables with the original DataFrame
merged_df = pd.concat([application_df, dummies], axis=1)

# Display the merged DataFrame
merged_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = merged_df.drop('IS_SUCCESSFUL', axis=1).values
y = merged_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [12]:
# Select continuous columns to scale
continuous_columns = ['STATUS', 'ASK_AMT']

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler on the training data
scaler.fit(X_train[:, [merged_df.columns.get_loc(col) for col in continuous_columns]])

# Scale the continuous features in the dataset
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[:, [merged_df.columns.get_loc(col) for col in continuous_columns]] = scaler.transform(X_train[:, [merged_df.columns.get_loc(col) for col in continuous_columns]])
X_test_scaled[:, [merged_df.columns.get_loc(col) for col in continuous_columns]] = scaler.transform(X_test[:, [merged_df.columns.get_loc(col) for col in continuous_columns]])

# Now you have X_train_scaled, X_test_scaled, y_train, and y_test with scaled continuous features

## Compile, Train and Evaluate the Model



In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dim = X_train_scaled.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(21, activation='relu', input_dim=input_dim))

# Second hidden layer
nn.add(Dense(14, activation='relu'))

# Third hidden layer
nn.add(Dense(7, activation='relu'))

# Output layer
nn.add(Dense(1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 21)                903       
                                                                 
 dense_1 (Dense)             (None, 14)                308       
                                                                 
 dense_2 (Dense)             (None, 7)                 105       
                                                                 
 dense_3 (Dense)             (None, 1)                 8         
                                                                 
Total params: 1,324
Trainable params: 1,324
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

Epoch 1/50
804/804 [==============================] - 8s 6ms/step - loss: 0.5923 - accuracy: 0.7081 - val_loss: 0.5655 - val_accuracy: 0.7219
Epoch 2/50
804/804 [==============================] - 6s 8ms/step - loss: 0.5601 - accuracy: 0.7256 - val_loss: 0.5624 - val_accuracy: 0.7230
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5562 - accuracy: 0.7273 - val_loss: 0.5634 - val_accuracy: 0.7203
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5551 - accuracy: 0.7270 - val_loss: 0.5567 - val_accuracy: 0.7278
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5530 - accuracy: 0.7282 - val_loss: 0.5562 - val_accuracy: 0.7216
Epoch 6/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5522 - accuracy: 0.7282 - val_loss: 0.5568 - val_accuracy: 0.7266
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5510 - accuracy: 0.7304 - val_loss: 0.5534 - val_accuracy: 0.7273
Epoch 

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5528 - accuracy: 0.7268 - 519ms/epoch - 2ms/step
Loss: 0.5528187155723572, Accuracy: 0.7267638444900513


In [17]:
from google.colab import drive
import os

# Create a directory and mount Google Drive
drive.mount('/content/drive')

# Specify the directory path in your Google Drive
drive_path = '/content/drive/My Drive/my_models'  # Change this to your desired directory path

# Make sure the specified directory exists
os.makedirs(drive_path, exist_ok=True)

# Export your model to an HDF5 file in the specified directory
nn.save(os.path.join(drive_path, 'AlphabetSoupCharityOptimization.h5'))

print("Model saved to Google Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved to Google Drive


In [18]:
#original accuracy: 0.7298

#OPTIMIZATION TEST #1: Reduced the hidden nodes to 10, 8<--Accuracy: 0.7262 (-0.0036)
#OPTIMIZATION TEST #2:  Added another hidden layer at 6<--accuracy: 0.7280 (-0.0018)
#OPTIMIZATION TEST #3: Keep the layers and nodes the same and increase the epochs to 100<--accuracy: 0.7298 (back to original level)
#OPTIMIZATION TEST #4: Change the nodes back to 21, 14, 7 and run with epoch =100<---accuracy: 0.7224 (-0.0074)
#OPTIMIZATION TEST #5: Keep everything the same and change the validation_data=(X_test_scaled, y_test)<---accuracy: 0.7263 (-0.0035)
#OPTIMIZATION TEST #6: Keep previous changes but change epochs back to 50<---accuracy: 0.7268 (-0.003)